# Загрузка данных анкет и проведение МАИ, анализ полученных результатов
Данные: заполненные анкеты экспертов (таблицы Excel) 

In [1]:
import re
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from random import randint

In [2]:
df = pd.read_excel(r'C:\Users\user\аварии\Экс11.xlsx')
df.head()

,Тип нарушения,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,Документация,1.000000,3.000000,3.0,5.000000,3.0,5.0,3
1,Экспертиза промышленной безопасности,0.333333,1.000000,1.0,7.000000,3.0,5.0,3
2,Готовность к ликвидации последствий аварий и и...,0.333333,1.000000,1.0,5.000000,1.0,5.0,3
3,Оформление информационными знаками и знаками б...,0.200000,0.142857,0.2,1.000000,3.0,1.0,3
4,Содержание объектов и оборудования,0.333333,0.333333,1.0,0.333333,1.0,7.0,1


In [3]:
print('Число строк и столбцов в наборе данных:\n', df.shape)

Число строк и столбцов в наборе данных:
 (7, 8)


In [4]:
# типы столбцов 
df.dtypes

Тип нарушения                                                 object
Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала         int64
dtype: object

In [5]:
#удаление столбца типа object для дальнейшей работы с фреймом 
df.pop('Тип нарушения') 
print(df)

   Документация   Экспертиза промышленной безопасности  \
0       1.000000                              3.000000   
1       0.333333                              1.000000   
2       0.333333                              1.000000   
3       0.200000                              0.142857   
4       0.333333                              0.333333   
5       0.200000                              0.200000   
6       0.333333                              0.333333   

   Готовность к ликвидации последствий аварий и инцидентов  \
0                                           3.000000         
1                                           1.000000         
2                                           1.000000         
3                                           0.200000         
4                                           1.000000         
5                                           0.200000         
6                                           0.333333         

   Оформление информационными знаками 

In [6]:
# типы столбцов 
df.dtypes

Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала         int64
dtype: object

In [7]:
# также меням тип данных Документации
df['Документация '] = df['Документация '].astype(float)

In [8]:
df.loc[:, df.columns[df.dtypes == 'float64']].head(7)

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ)
0,1.000000,3.000000,3.000000,5.000000,3.000000,5.000000
1,0.333333,1.000000,1.000000,7.000000,3.000000,5.000000
2,0.333333,1.000000,1.000000,5.000000,1.000000,5.000000
3,0.200000,0.142857,0.200000,1.000000,3.000000,1.000000
4,0.333333,0.333333,1.000000,0.333333,1.000000,7.000000
5,0.200000,0.200000,0.200000,1.000000,0.142857,1.000000
6,0.333333,0.333333,0.333333,0.333333,1.000000,0.333333


In [9]:
DF_1=df[:].copy()

In [10]:
#Найдем сумму всех столбцов, для того чтобы в дальнейшем пересчитать полученные значения
df.sum ()

Документация                                                  2.733333
Экспертиза промышленной безопасности                          6.009524
Готовность к ликвидации последствий аварий и инцидентов       6.733333
Оформление информационными знаками и знаками безопасности    19.666667
Содержание объектов и оборудования                           12.142857
Охранные зоны (ОЗ)                                           24.333333
Организация рабочего процесса и безопасности персонала       17.000000
dtype: float64

In [11]:
# Считаем значения для нашей матрицы, путем деления каждого значения в столбце на сумму столбца
df['new_Документация'] = df['Документация '].astype(float).div(df['Документация '].sum())
df['new_Экспертиза'] = df['Экспертиза промышленной безопасности'].astype(float).div(df['Экспертиза промышленной безопасности'].sum())
df['new_Готовность'] = df['Готовность к ликвидации последствий аварий и инцидентов'].astype(float).div(df['Готовность к ликвидации последствий аварий и инцидентов'].sum())
df['new_Оформление'] = df['Оформление информационными знаками и знаками безопасности'].astype(float).div(df['Оформление информационными знаками и знаками безопасности'].sum())
df['new_Содержание'] = df['Содержание объектов и оборудования '].astype(float).div(df['Содержание объектов и оборудования '].sum())
df['new_ОЗ'] = df['Охранные зоны (ОЗ)'].astype(float).div(df['Охранные зоны (ОЗ)'].sum())
df['new_Организация'] = df['Организация рабочего процесса и безопасности персонала'].astype(float).div(df['Организация рабочего процесса и безопасности персонала'].sum())

In [12]:
df.head()

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,1.000000,3.000000,3.0,5.000000,3.0,5.0,3,0.365854,0.499208,0.445545,0.254237,0.247059,0.205479,0.176471
1,0.333333,1.000000,1.0,7.000000,3.0,5.0,3,0.121951,0.166403,0.148515,0.355932,0.247059,0.205479,0.176471
2,0.333333,1.000000,1.0,5.000000,1.0,5.0,3,0.121951,0.166403,0.148515,0.254237,0.082353,0.205479,0.176471
3,0.200000,0.142857,0.2,1.000000,3.0,1.0,3,0.073171,0.023772,0.029703,0.050847,0.247059,0.041096,0.176471
4,0.333333,0.333333,1.0,0.333333,1.0,7.0,1,0.121951,0.055468,0.148515,0.016949,0.082353,0.287671,0.058824


In [13]:
# Получили новую таблицу теперь отделим значения нашей матрицы от основной таблицы
df_new = df[['new_Документация', 'new_Экспертиза', 'new_Готовность', 'new_Оформление', 'new_Содержание', 'new_ОЗ', 'new_Организация']]
df_new

,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,0.365854,0.499208,0.445545,0.254237,0.247059,0.205479,0.176471
1,0.121951,0.166403,0.148515,0.355932,0.247059,0.205479,0.176471
2,0.121951,0.166403,0.148515,0.254237,0.082353,0.205479,0.176471
3,0.073171,0.023772,0.029703,0.050847,0.247059,0.041096,0.176471
4,0.121951,0.055468,0.148515,0.016949,0.082353,0.287671,0.058824
5,0.073171,0.033281,0.029703,0.050847,0.011765,0.041096,0.176471
6,0.121951,0.055468,0.049505,0.016949,0.082353,0.013699,0.058824


In [14]:
#Вычислим средние значение по строке и запишем полученный результот в mean
df_new['mean'] = df_new.mean(axis=1)
df_new.head()

C:\Users\user\anaconda3\envs\envpy3.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация,mean
0,0.365854,0.499208,0.445545,0.254237,0.247059,0.205479,0.176471,0.313407
1,0.121951,0.166403,0.148515,0.355932,0.247059,0.205479,0.176471,0.203116
2,0.121951,0.166403,0.148515,0.254237,0.082353,0.205479,0.176471,0.165058
3,0.073171,0.023772,0.029703,0.050847,0.247059,0.041096,0.176471,0.091731
4,0.121951,0.055468,0.148515,0.016949,0.082353,0.287671,0.058824,0.110247


In [15]:
#проверка, среднее значение в сумме должно быть равно 1
df_new['mean'].sum()

1.0

In [16]:
# Далее проведем проверку на согласованность, 
#для этого создадим из нашей полученной выше таблицы матрицы для дальнейшего их перемножения
df_new8 =df_new['mean']

# удаляем из нашей таблицы значениея столбцы mean
#df_new.pop('mean')

df_new1 =DF_1.loc[0]

df_new2 =DF_1.loc[1]

df_new3 =DF_1.loc[2]

df_new4 =DF_1.loc[3]

df_new5 =DF_1.loc[4]

df_new6 =DF_1.loc[5]

df_new7 =DF_1.loc[6]

In [17]:
#Проверка на согласованность
res1 = np.dot(df_new1,df_new8)
print(res1)
res2 = np.dot(df_new2,df_new8)
print(res2)
res3 = np.dot(df_new3,df_new8)
print(res3)
res4 = np.dot(df_new4,df_new8)
print(res4)
res5 = np.dot(df_new5,df_new8)
print(res5)
res6 = np.dot(df_new6,df_new8)
print(res6)
res7 = np.dot(df_new7,df_new8)
print(res7)

2.6755997519573143
1.9137756695359145
1.509818900550437
0.7775505804029523
0.9513538820281098
0.47416517414695114
0.444807463888227


In [18]:
sum_new = res1 + res2 + res3 + res4 + res5 + res6 + res7
print(sum_new)

8.747071422509904


In [19]:
# Вычислем значения CI, RI, CR
CI = (sum_new - 7)/(7-1)
print(CI)
RI = (1.98*(7-2))/7
print(RI)
CR = CI/RI
print(CR)

0.2911785704183174
1.4142857142857144
0.20588383766951734


In [20]:
# Получилось, что значение CR=0.21, что большее 0,1 значит матрицу нельзя считать согласованной
# Проделаем аналогичные действия с другими экспертами